In [1]:
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#import gdal
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
import rasterio
import rasterio.plot
import rasterio.mask # an important package which allows you to only extract points within your shapefile
import rasterstats # gives us raster summaries within a shapefile
from rasterstats import zonal_stats # gives us raster summaries within a shapefile

#import osmnx as ox # open streetmap package
os.getcwd()

'C:\\Users\\leogo\\OneDrive\\Desktop\\PhD\\Analysis\\data\\rhomis_data'

In [3]:
#os.chdir('../../')
os.getcwd()

'C:\\Users\\leogo\\OneDrive\\Desktop\\PhD\\Analysis'

In [4]:
world_grid=gpd.read_file(os.path.join('data', 'shapefiles', 'world_grid_100.0km'))
geometry=world_grid.values #contains country name, metadata and

In [151]:
#https://gis.stackexchange.com/questions/297076/how-to-calculate-mean-value-of-a-raster-for-each-polygon-in-a-shapefile

filepath=os.path.join("data", "raster", "ClimAfr21_education_index", "ClimAfr21_education index.tif")
#print the metadata of the tif (rastwer) file
with rasterio.open(filepath) as src:
    # can check the profile and type of src, but effectively is an open file command
    affine = src.transform # transforms the raster dataset into the coordinate reference syste
    raster_array=rasterio.open(filepath).read(1) # why is this "1" here?
    raster_statistics_for_grid = pd.DataFrame(zonal_stats(world_grid, raster_array, affine=affine,nodata=None)) # calculates raster statistics within boundary of shapefile

    
    


In [152]:
#no_Na=raster_statistics_for_grid.loc[raster_statistics_for_grid.iloc[:,0]>(0),:]
#no_Na.iloc[:,0].plot.hist(bins=20)


grid_raster_merge=raster_statistics_for_grid

In [153]:
#grid_raster_merge.columns=
#mapped_value="education"
#grid_raster_merge=grid_raster_merge.replace({-9999.0:"NaN"})
#grid_raster_merge=grid_raster_merge.rename({"min":"min_"+mapped_value})
#grid_raster_merge=grid_raster_merge.rename({"max":"max_"+mapped_value})
#grid_raster_merge=grid_raster_merge.rename({"mean":"mean_"+mapped_value})
#grid_raster_merge=grid_raster_merge.rename({"count":"count_"+mapped_value})

#grid_raster_merge.head()
grid_raster_merge

,min,max,mean,count
0,-3.402823e+38,-3.402823e+38,-3.402823e+38,11664
1,-3.402823e+38,-3.402823e+38,-3.402823e+38,11664
2,-3.402823e+38,-3.402823e+38,-3.402823e+38,11664
3,-3.402823e+38,-3.402823e+38,-3.402823e+38,11664
4,-3.402823e+38,-3.402823e+38,-3.402823e+38,11664
...,...,...,...,...
51239,NaN,NaN,NaN,0
51240,NaN,NaN,NaN,0
51241,NaN,NaN,NaN,0
51242,NaN,NaN,NaN,0


In [133]:
def is_float(x):
    try: 
        return(float(x))
    except:
        return np.nan
def replace_negatives_with_na(x): 
    if x<0:
        x=np.nan
    return x;
def find_all_na_in_raster(dataframe, columns):
    dataframe.loc[:,columns]=dataframe.loc[:,columns].applymap(is_float)
    dataframe.loc[:,columns]=dataframe.loc[:,columns].applymap(replace_negatives_with_na)
    return dataframe;

grid_raster_merge=find_all_na_in_raster(dataframe=grid_raster_merge, columns=["min", "max", "mean"])

nan

In [147]:

#subsetting_non_na_values=grid_raster_merge.loc[:,"mean"]!="NaN"

subsetting_non_na_values=grid_raster_merge.loc[:,"mean"].notna()
subsetting_non_na_values

grid_raster_merge=grid_raster_merge.loc[subsetting_non_na_values,:]


In [150]:
grid_raster_merge

,min,max,mean,count
85,0.509599,0.509599,0.509599,11664
89,0.509599,0.509599,0.509599,11664
91,0.509599,0.509599,0.509599,11664
93,0.509599,0.509599,0.509599,11664
95,0.509599,0.509599,0.509599,11664
...,...,...,...,...
29481,1.186330,1.272005,1.218896,14256
29485,1.048108,1.272005,1.236952,14256
29489,1.048108,1.244029,1.151459,14256
29497,1.295270,1.295270,1.295270,14256
